In [147]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineG

In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 17.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    LlamaForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model
import torch
import numpy as np




In [148]:
import pandas as pd

# Load the dataset
data = pd.read_json('./combined_shuffled.json')

# Display the first few rows
print(data.head())

# Check data types of each column
print(data.dtypes)

                                               input  \
0    What are some fun activities to do in New York?   
1  Can you find 4-star hotels with a swimming poo...   
2  I need to plan a trip from Toronto (YYZ) to Li...   
3     What are some fun things to do in Mexico City?   
4  I need a flight from AER to BKK on 2023-11-15 ...   

                                              output  
0   {'action': ['activities'], 'destination': 'NYC'}  
1  {'action': ['search_hotels'], 'action_input': ...  
2  {'action': ['search_flights', 'search_hotels',...  
3   {'action': ['activities'], 'destination': 'MEX'}  
4  {'action': ['search_flights', 'activities'], '...  
input     object
output    object
dtype: object


In [149]:
import json

# Ensure 'output' is of type string by converting dictionaries to JSON strings
data['output'] = data['output'].apply(json.dumps)

# Verify the conversion
print(data.head())
print(data.dtypes)

                                               input  \
0    What are some fun activities to do in New York?   
1  Can you find 4-star hotels with a swimming poo...   
2  I need to plan a trip from Toronto (YYZ) to Li...   
3     What are some fun things to do in Mexico City?   
4  I need a flight from AER to BKK on 2023-11-15 ...   

                                              output  
0   {"action": ["activities"], "destination": "NYC"}  
1  {"action": ["search_hotels"], "action_input": ...  
2  {"action": ["search_flights", "search_hotels",...  
3   {"action": ["activities"], "destination": "MEX"}  
4  {"action": ["search_flights", "activities"], "...  
input     object
output    object
dtype: object


In [150]:
from datasets import Dataset, DatasetDict, Features, Value

# Select the first 700 entries (adjust as needed)
dataset_pandas = data.iloc[:5]

dataset_hf = Dataset.from_pandas(dataset_pandas)

In [ ]:
# Perform train-test split
split_dataset = dataset_hf.train_test_split(test_size=0.2)

dataset = DatasetDict({
    'train': split_dataset['train'],
    'validation': split_dataset['test']
})

# 2. Initialize the Tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 3. Preprocess the Data
def preprocess_function(examples):
    inputs = examples['input']
    outputs = examples['output']
    formatted_inputs = [f"User: {inp}\nAssistant: {out}" for inp, out in zip(inputs, outputs)]
    return tokenizer(formatted_inputs, truncation=True, padding='max_length', max_length=512)

# Apply the preprocessing to the dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# 4. Load the Pre-trained LLaMA Model
model = LlamaForCausalLM.from_pretrained(
    'meta-llama/Llama-3.2-1B',
    load_in_8bit=False,
    torch_dtype=torch.float32,  # Ensure float32 is used
    device_map='auto'             # auto
)

# Verify that all model parameters are in float32
for param in model.parameters():
    if param.dtype != torch.float32:
        print(f"Parameter {param.name} is not in float32!")

# 5. Configure LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

trainable params: 1,703,936 || all params: 1,237,518,336 || trainable%: 0.1377


In [ ]:
training_args = TrainingArguments(
    output_dir='../model',          # Directory to save the fine-tuned model
    num_train_epochs=1,             # Number of training epochs
    per_device_train_batch_size=4,  # Batch size per device during training
    per_device_eval_batch_size=8,   # Batch size for evaluation
    gradient_accumulation_steps=4,  # Accumulate gradients to simulate larger batch size
    learning_rate=2e-4,             # Learning rate
    weight_decay=0.01,              # Weight decay for optimizer
    evaluation_strategy="epoch",    # Evaluate at the end of each epoch
    save_strategy="epoch",          # Save checkpoint at the end of each epoch
    logging_dir='./logs',           # Directory for logs
    logging_steps=10,               # Log every 10 steps
    save_total_limit=2,             # Maximum number of checkpoints to save
    fp16=False,                     # Disable mixed precision
    no_cuda=True,                   # Disable CUDA/MPS and use CPU
    load_best_model_at_end=True,    # Load the best model when finished training
    metric_for_best_model="loss",   # Use loss to evaluate the best model
    greater_is_better=False,        # Lower loss is better
    save_safetensors=False           # **Disable safetensors**
)

# 7. Initialize the Data Collator
# Using Hugging Face's built-in DataCollator to handle padding and other preprocessing
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # Set to False for causal language modeling
)

# 8. Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1583: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(
<ipython-input-18-17fa78a568a9>:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# 9. Start Training
trainer_stats = trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,No log,1.871052


In [ ]:
# 10. Save the Fine-Tuned Model and Tokenizer
trainer.save_model('./lora_llama_finetuned/model_finetuned')  # Saves both base model and adapters
tokenizer.save_pretrained('./lora_llama_finetuned/tokenizer_finetuned')
model.save_pretrained('./lora_llama_finetuned/model_finetuned')

# check you can load it

In [ ]:
from transformers import LlamaForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

In [ ]:
# Paths
model_path = './lora_llama_finetuned/model_finetuned'      # Path to the LoRA adapters
base_model_path = './lora_llama_finetuned/base_model'      # Path to the base model (if saved separately)
tokenizer_path = './lora_llama_finetuned/tokenizer_finetuned'

# Load the base model
base_model = LlamaForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-1B",
    torch_dtype=torch.float32,
    device_map='auto'  # Automatically maps to available devices
)

# Load the LoRA adapters onto the base model
model_loaded = PeftModel.from_pretrained(
    base_model,
    model_path
)

# Load the tokenizer
tokenizer_loaded = AutoTokenizer.from_pretrained(tokenizer_path)

# Ensure the pad token is set (if not already done during training)
if tokenizer_loaded.pad_token is None:
    tokenizer_loaded.pad_token = tokenizer_loaded.eos_token

# Move the model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_loaded.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048)
        (layers): ModuleList(
          (0-15): 16 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linea

In [ ]:
model_loaded.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048)
        (layers): ModuleList(
          (0-15): 16 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linea

In [ ]:
# Define a formatted prompt consistent with training
prompt = "what is your name"

# Tokenize the input
inputs = tokenizer_loaded(prompt, return_tensors="pt").to(device)

# Generate a response
with torch.no_grad():
    output_tokens = model_loaded.generate(
        **inputs,
        max_length=250,        # Increase if you need a longer answer
        temperature=0.1,       # Adjust temperature for more or less randomness
        top_p=0.9,             # Adjust top_p for nucleus sampling
        do_sample=True         # Set to False for deterministic output (greedy)
    )

# Decode the output tokens to text
response = tokenizer_loaded.decode(output_tokens[0], skip_special_tokens=True)

print(response)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


what is your name? what is your age? what is your gender? what is your nationality? what is your religion? what is your occupation? what is your education? what is your marital status? what is your family status? what is your income? what is your profession? what is your profession? what is your profession? what is your profession? what is your profession? what is your profession? what is your profession? what is your profession? what is your profession? what is your profession? what is your profession? what is your profession? what is your profession? what is your profession? what is your profession? what is your profession? what is your profession? what is your profession? what is your profession? what is your profession? what is your profession? what is your profession? what is your profession? what is your profession? what is your profession? what is your profession? what is your profession? what is your profession? what is your profession? what is your profession? what is your pro

In [ ]:
response

'User: I need a flight from london to CBR on 2023-12-15 for 2 adults and 1 child in business class, non-stop only.\nAssistant: I need a flight from london to CBR on 2023-12-15 for 2 adults and 1 child in business class, non-stop only.\nI need a flight from london to CBR on 2023-12-15 for 2 adults and 1 child in business class, non-stop only.\nI need a flight from london to CBR on 2023-12-15 for 2 adults and 1 child in business class, non-stop only.\nI need a flight from london to CBR on 2023-12-15 for 2 adults and 1 child in business class, non-stop only.\nI need a flight from london to CBR on 2023-12-15 for 2 adults and 1 child in business class, non-stop only.\nI need a flight from london to CBR on 2023-12-15 for 2 adults and 1 child in business class, non-stop only.\nI need a flight from london to CBR on 2023-'

# YOUTUBE

In [ ]:
pip install -U bitsandbytes

In [165]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, LlamaForCausalLM

model = LlamaForCausalLM.from_pretrained(
    'meta-llama/Llama-3.2-3B',
    load_in_8bit=False,
    torch_dtype=torch.float32,  # Ensure float32 is used
    device_map='auto'             # auto
)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B")



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [166]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

In [167]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [168]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16, #attention heads
    lora_alpha=32, #alpha scaling
    # target_modules=["q_proj", "v_proj"], #if you know the
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM" # set this for CLM or Seq2Seq
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4587520 || all params: 3217337344 || trainable%: 0.1425874724810952


In [169]:
with open('./combined_shuffled.json', 'r') as file:
    data = json.load(file)

# Filter specific keys from action_input
filtered_data = []
for item in data:
  try:
    # Extract only the desired keys from action_input
    filtered_action_input = {
        key: item['output']['action_input'][key]
        for key in ['origin', 'destination', 'departure_date', 'adults']
    }

    # Construct the new dictionary
    filtered_item = {
        'input': item['input'],
        'output': {
            'action': item['output']['action'],
            'action_input': filtered_action_input
        }
    }
    filtered_data.append(filtered_item)
  except:
    continue

def merge_columns(example):
    example["prediction"] = example["input"] + " -> " + str(example["output"])
    return example

dataset = dataset.map(merge_columns)

data = dataset.map(lambda samples: tokenizer(samples['prediction']), batched=True)

In [ ]:
dataset = Dataset.from_list(filtered_data)

In [171]:
dataset['prediction'][0]

"I need to plan a trip from Toronto (YYZ) to Lima (LIM) starting on 2023-12-10. I want to book flights, hotels, and activities for 2 adults and 1 child, preferably in business class. The hotel should have a swimming pool and WiFi, and I would like options for non-stop flights only. -> {'action': ['search_flights', 'search_hotels', 'activities'], 'action_input': {'adults': 2, 'departure_date': '2023-12-10', 'destination': 'LIM', 'origin': 'YYZ'}}"

In [172]:
data = dataset.map(lambda samples: tokenizer(samples['prediction']), batched=True)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [173]:
data

Dataset({
    features: ['input', 'output', 'prediction', 'input_ids', 'attention_mask'],
    num_rows: 400
})

In [174]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

tokenizer.pad_token = tokenizer.eos_token

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()

Step,Training Loss
1,2.114400
2,2.166900
3,2.193000


Step,Training Loss
1,2.114400
2,2.166900
3,2.193000
4,2.091600
5,2.107400
6,2.095100
7,2.017800
8,1.980400
9,1.886200
10,1.865900


TrainOutput(global_step=100, training_loss=0.8370114234089852, metrics={'train_runtime': 16175.3338, 'train_samples_per_second': 0.099, 'train_steps_per_second': 0.006, 'total_flos': 3337385146097664.0, 'train_loss': 0.8370114234089852, 'epoch': 4.0})

In [ ]:
model.push_to_hub("llama-32-3B-parser",
                  use_auth_token=True,
                  commit_message="basic training",
                  private=False)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:894: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/18.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jaeho127/llama-32-3B-parser/commit/02d6c507cfe9799362a3d746f9324a05233f960d', commit_message='basic training', commit_description='', oid='02d6c507cfe9799362a3d746f9324a05233f960d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/jaeho127/llama-32-3B-parser', endpoint='https://huggingface.co', repo_type='model', repo_id='jaeho127/llama-32-3B-parser'), pr_revision=None, pr_num=None)

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "jaeho127/llama-32-3B-parser"
config = PeftConfig.from_pretrained(peft_model_id)
model = LlamaForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=False, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

adapter_config.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/18.4M [00:00<?, ?B/s]

In [ ]:
batch = tokenizer("I want a trip to zimbabwe to london on January 1 2025. Give me hotel information too -> ", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=50)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

<ipython-input-142-3dfb02c156c2>:3: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.




 I want a trip to zimbabwe to london on January 1 2025. Give me hotel information too ->  {'action': ['search_flights','search_hotels'], 'action_input': {'adults': 1, 'departure_date': '2025-01-01', 'destination': 'LON', 'origin': 'HAR'}}


# 1B model

In [175]:
with open('./combined_shuffled.json', 'r') as file:
    data = json.load(file)

# Filter specific keys from action_input
filtered_data = []
for item in data:
  try:
    # Extract only the desired keys from action_input
    filtered_action_input = {
        key: item['output']['action_input'][key]
        for key in ['origin', 'destination', 'departure_date', 'adults']
    }

    # Construct the new dictionary
    filtered_item = {
        'input': item['input'],
        'output': {
            'action': item['output']['action'],
            'action_input': filtered_action_input
        }
    }
    filtered_data.append(filtered_item)
  except:
    continue

def merge_columns(example):
    example["prediction"] = example["input"] + " -> " + str(example["output"])
    return example

dataset = dataset.map(merge_columns)

data = dataset.map(lambda samples: tokenizer(samples['prediction']), batched=True)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [196]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, LlamaForCausalLM

model = LlamaForCausalLM.from_pretrained(
    'meta-llama/Llama-3.2-1B',
    load_in_8bit=False,
    torch_dtype=torch.float32,  # Ensure float32 is used
    device_map='auto'             # auto
)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")

for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )



from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16, #attention heads
    lora_alpha=32, #alpha scaling
    # target_modules=["q_proj", "v_proj"], #if you know the
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM" # set this for CLM or Seq2Seq
)

model = get_peft_model(model, config)
print_trainable_parameters(model)



trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

tokenizer.pad_token = tokenizer.eos_token

max_steps is given, it will override any value given in num_train_epochs


trainable params: 1703936 || all params: 1237518336 || trainable%: 0.13768975783482856


In [177]:
trainer.train()

Step,Training Loss
1,2.246900
2,2.298100
3,2.331200
4,2.203700
5,2.230900
6,2.242700
7,2.136700
8,2.123200
9,2.036600
10,2.016800


TrainOutput(global_step=100, training_loss=0.9437171375751495, metrics={'train_runtime': 6481.8305, 'train_samples_per_second': 0.247, 'train_steps_per_second': 0.015, 'total_flos': 1152342945939456.0, 'train_loss': 0.9437171375751495, 'epoch': 4.0})

In [178]:
model.push_to_hub("llama-32-1B-parser",
                  use_auth_token=True,
                  commit_message="basic training",
                  private=False)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:894: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/6.82M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jaeho127/llama-32-1B-parser/commit/c746740be3d09f3774249e5c88423528e4c331f0', commit_message='basic training', commit_description='', oid='c746740be3d09f3774249e5c88423528e4c331f0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/jaeho127/llama-32-1B-parser', endpoint='https://huggingface.co', repo_type='model', repo_id='jaeho127/llama-32-1B-parser'), pr_revision=None, pr_num=None)

In [179]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "jaeho127/llama-32-1B-parser"
config = PeftConfig.from_pretrained(peft_model_id)
model = LlamaForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=False, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

adapter_model.safetensors:   0%|          | 0.00/6.82M [00:00<?, ?B/s]

In [180]:
batch = tokenizer("I want a trip to zimbabwe to london on January 1 2025. Give me hotel information too -> ", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=50)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

<ipython-input-180-3dfb02c156c2>:3: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.




 I want a trip to zimbabwe to london on January 1 2025. Give me hotel information too ->  {'action': ['search_flights','search_hotels'], 'action_input': {'adults': 1, 'departure_date': '2025-01-01', 'destination': 'LHR', 'origin': 'ZIH'}}


# Small Model

In [181]:
with open('./combined_shuffled.json', 'r') as file:
    data = json.load(file)

# Filter specific keys from action_input
filtered_data = []
for item in data:
  try:
    # Extract only the desired keys from action_input
    filtered_action_input = {
        key: item['output']['action_input'][key]
        for key in ['origin', 'destination', 'departure_date', 'adults']
    }

    # Construct the new dictionary
    filtered_item = {
        'input': item['input'],
        'output': {
            'action': item['output']['action'],
            'action_input': filtered_action_input
        }
    }
    filtered_data.append(filtered_item)
  except:
    continue

def merge_columns(example):
    example["prediction"] = example["input"] + " -> " + str(example["output"])
    return example

dataset = dataset.map(merge_columns)

data = dataset.map(lambda samples: tokenizer(samples['prediction']), batched=True)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [202]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, LlamaForCausalLM
import transformers

pretrained = 'HuggingFaceTB/SmolLM-135M'

model = LlamaForCausalLM.from_pretrained(
    pretrained,
    load_in_8bit=False,
    torch_dtype=torch.float16,  # Ensure float32 is used
    device_map='auto'             # auto
)

tokenizer = AutoTokenizer.from_pretrained(pretrained)

for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )



from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=4, #attention heads
    lora_alpha=32, #alpha scaling
    # target_modules=["q_proj", "v_proj"], #if you know the
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM" # set this for CLM or Seq2Seq
)

model = get_peft_model(model, config)
print_trainable_parameters(model)



trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        max_steps=100,
        learning_rate=2e-4,
        fp16=False,
        logging_steps=1,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)


model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

tokenizer.pad_token = tokenizer.eos_token

max_steps is given, it will override any value given in num_train_epochs


trainable params: 230400 || all params: 134745408 || trainable%: 0.17098912936610056


In [203]:
trainer.train()

IndexError: index out of range in self

In [194]:
pretrained = 'HuggingFaceTB/SmolLM-135M'

model = LlamaForCausalLM.from_pretrained(
    pretrained,
    load_in_8bit=False,
    torch_dtype=torch.float32,  # Ensure float32 is used
    device_map='auto'             # auto
)

tokenizer = AutoTokenizer.from_pretrained(pretrained)

import torch

# Example input text
input_text = "Once upon a time in a land far, far away"

# Tokenize the input text
inputs = tokenizer(input_text, return_tensors="pt")  # Ensure tensors are on the same device as the model

# Generate output using the pretrained model
outputs = model.generate(
    inputs["input_ids"],
    max_length=50,  # Maximum length of the generated text
    num_return_sequences=1,  # Number of sequences to generate
    do_sample=True,  # Use sampling for diversity
    top_k=50,  # Limit the sampling pool to top-k tokens
    top_p=0.95,  # Use nucleus sampling with a probability threshold
    temperature=0.7  # Adjust creativity (lower = more focused, higher = more random)
)

# Decode the generated text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated Text:", generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated Text: Once upon a time in a land far, far away, there was a beautiful village called Harmonyville. In Harmonyville, everyone lived together happily and helped each other out. But sometimes, things would go wrong. One day, a terrible storm hit the village


# Method with system prompt

In [2]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [115]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [116]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    # target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
    #                   "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [117]:
import json
from datasets import Dataset, DatasetDict, Features, Value

with open('./combined_shuffled.json', 'r') as file:
    data = json.load(file)

# # Filter specific keys from action_input
# filtered_data = []
# for item in data:
#   try:
#     # Extract only the desired keys from action_input
#     filtered_action_input = {
#         key: item['output']['action_input'][key]
#         for key in ['origin', 'destination', 'departure_date', 'adults']
#     }

#     # Construct the new dictionary
#     filtered_item = {
#         'input': item['input'],
#         'output': {
#             'action': item['output']['action'],
#             'action_input': filtered_action_input
#         }
#     }
#     filtered_data.append(filtered_item)
#   except:
#     continue

# def merge_columns(example):
#     example["prediction"] = example["input"] + " -> " + str(example["output"])
#     return example
# dataset = Dataset.from_list(filtered_data)
# dataset = dataset.map(merge_columns)
# data = dataset.map(lambda samples: tokenizer(samples['prediction']), batched=True)

In [118]:
def convert_data(data_list):
    converted_data = []
    for data in data_list:
        converted_data.append([
            {"role": "system", "content": "You are a parser that parses user input"},
            {"role": "user", "content": data['input']},
            {"role": "assistant", "content": str(data['output'])}
        ])
    return converted_data

converted_data_list = convert_data(data)

converted_data_list = convert_data(filtered_data)

In [122]:
# Convert to conversations structure
def convert_to_conversations(data_list):
    conversations = []
    for entry in data_list:
        conversations.append({"conversations": entry})
    return conversations

# Convert the data
conversations_data = convert_to_conversations(converted_data_list)

In [123]:
def ensure_consistent_conversations(data_list):
    processed = []
    for entry in data_list:
        if isinstance(entry, list) and all(isinstance(e, dict) for e in entry):
            processed.append({"conversations": entry})
        else:
            # Handle unexpected or missing data by setting a default value
            processed.append({"conversations": []})
    return processed

# Convert the data
conversations_data = convert_to_conversations(converted_data_list)

In [124]:
dataset_sloth = Dataset.from_list(conversations_data)

In [125]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from unsloth.chat_templates import standardize_sharegpt

dataset = standardize_sharegpt(dataset_sloth)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Standardizing format:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [126]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map (num_proc=2):   0%|          | 0/400 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [127]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 400 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 11,272,192


Step,Training Loss
1,1.556200
2,1.633900
3,1.520300
4,1.256000
5,1.038700
6,0.636500
7,0.385900
8,0.155300
9,0.080200
10,0.063400


In [132]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "I want a flight from paris to london on 2025-01-01 with five adults"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

["<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nI want a flight from paris to london on 2025-01-01 with five adults<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n{'action': ['search_flights'], 'action_input': {'origin': 'PAR', 'destination': 'LON', 'departure_date': '2025-01-01', 'adults': 5}}<|eot_id|>"]

In [129]:
model.save_pretrained("lora_model_all_param")
tokenizer.save_pretrained("lora_model_all_param")

('lora_model_all_param/tokenizer_config.json',
 'lora_model_all_param/special_tokens_map.json',
 'lora_model_all_param/tokenizer.json')

In [130]:
if True: model.save_pretrained_gguf("model_more_param", tokenizer, quantization_method = "f16")

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 34.11 out of 50.99 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 16/16 [00:00<00:00, 45.00it/s]


Unsloth: Saving tokenizer... Done.
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['f16'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at model_more_param into f16 GGUF format.
The output location will be /content/model_more_param/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: model_more_param
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {32}
INFO:hf-to-gguf:gguf: loading model part 'model.safetensors'
INFO:hf-to-gguf:token_embd.weight,           torch.float16 --> F16, shape = {2048, 128256}

In [131]:
total_tunable_params = 0
for name, param in model.named_parameters():
    if param.requires_grad:
        total_tunable_params += param.numel()

print(f"Total Tunable Parameters: {total_tunable_params}")

Total Tunable Parameters: 11272192
